# 6 A climate component of a planner's decision problem

This notebook corresponds to section 6 in the paper.

To check the previous section:

[Section 5: Uncertainty aversion](sec5_UncertaintyAversion.ipynb)

In [29]:
%matplotlib inline
import pandas as pd
import numpy as np
from source.model import solve_hjb_y, solve_hjb_y_jump
from source.utilities import find_nearest_value
from source.simulation import simulate_jump
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['legend.frameon'] = False

from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.layouts import row
from bokeh.models import Span
output_notebook()

Loading BokehJS ...

# 1 Parameters
We use the following values for the parameters listed below throughout:

| Parameters | values |
| :---:| :---|
|$\delta$ |  0.01 |
|$\eta$ | 0.032 | 
|$\bar y$ | 2 |
|$\gamma_1$ | 0.00017675 |
|$\gamma_2$ | 0.0044|
|$\gamma_3^1$| 0.0000|
|$\gamma_3^2$| 0.0394|
|$\gamma_3^3$| 0.7706|
|$\varsigma'$| [0.0022, 0]|





# 2. Computation

Choose a set of uncertainty parameters $(\xi_b, \xi_p, \xi_a)$ and proceed with the following two step computation. We explore 144 climate models and 3 damage functions, *i.e.* $L=144$ and $M=3$.

## 2.1 Step I:
Solve the following HJB for $\gamma_3^m \in \{\gamma_3^1, \gamma_3^2, \gamma_3^3\}$ on $y \in [0,4]$ to get the corresponding $\phi_m(y)$ with $m = 1,2,3$:

$$
\begin{aligned}
0 = \max_{\tilde e} \min_{\omega^a_\ell : \sum_{\ell=1}^L \omega^a_\ell = 1}  &- \delta \phi_m(y) +  \eta \log\tilde e \\
& + \frac{1}{2} \left(\frac{d^2 \phi_m}{dy^2} + \frac{ (\eta - 1)}{\delta} \left(\gamma_2 + \gamma_3\mathbb{I}\{y>\bar y\} \right) \right)(\tilde e)^2 |\varsigma|^2  \\
& - \frac{1}{2\xi_b} \left[ \frac{d\phi_m}{dy}    + \frac{(\eta -1)}{\delta}(\gamma_1 + \gamma_2 y + \gamma_3 (y-\bar y)\mathbb{I}\{y > \bar y\})\right]^2 \cdot |\varsigma|^2 (\tilde e)^2 \\
\\
& + \sum_{\ell=1}^{L} \omega_\ell^a \left(\frac{d\phi_m}{dy}+ \frac{(\eta -1)}{\delta}(\gamma_1 + \gamma_2 y + \gamma_3^m (y - \bar y)\mathbb{I}\{y > \bar y\} ) \right)\theta_\ell \tilde e   \\
\\
& + \xi_a \sum_i \omega^a_\ell(\log \omega^a_\ell - \log \pi^a_\ell)
\end{aligned}
$$

## 2.2 Step II:
Given the above $\phi_1$, $\phi_2$ and $\phi_3$, solve the following HJB for $\phi(y)$ on $y \in [0, \bar y]$:

$$
\begin{aligned}
0 = \max_{\tilde e} \min_{\omega^a_\ell : \sum_{\ell=1}^L \omega^a_\ell = 1}  &- \delta \phi(y) +  \eta \log\tilde e \\
&+ \frac{1}{2} \left(\frac{d^2 \phi_m}{dy^2} + \frac{ (\eta - 1)}{\delta} \left(\gamma_2 + \gamma_3\mathbb{I}\{y>\bar y\} \right) \right)(\tilde e)^2 |\varsigma|^2 \\
\\
& - \frac{1}{2\xi_b} \left[ \frac{d\phi}{dy}    + \frac{(\eta -1)}{\delta}(\gamma_1 + \gamma_2 y + \gamma_3 (y-\bar y)\mathbb{I}\{y > \bar y\})\right]^2 \cdot |\varsigma|^2 (\tilde e)^2 \\
\\
&  + \sum_{\ell=1}^{L} \omega_\ell^a \left(\frac{d\phi}{dy}+ \frac{(\eta -1)}{\delta}(\gamma_1 + \gamma_2 y + \gamma_3^m (y - \bar y)\mathbb{I}\{y > \bar y\} ) \right)\theta_\ell \tilde e   \\
\\
 & + \xi_a \sum_{\ell=1}^L \omega^a_\ell(\log \omega^a_\ell - \log \pi^a_\ell)
\end{aligned}
$$

$$
s.t \quad \phi(\bar y) \approx - \xi_p \log \left( \sum_{m=1}^M {\pi^d_m} \exp ( - \frac{1}{\xi_p}\phi_m(\bar y) )\right)
$$

In [7]:
# Uncertainty parameters
ξ_w = 10_000
ξ_a = 1/100
ξ_p = 5

# Preference
η = .032
δ = .01

# Climate sensitivity
θ_list = pd.read_csv('data/model144.csv', header=None).to_numpy()[:, 0]/1000.
πc_o = np.ones_like(θ_list)/len(θ_list)

# Damage function
σ_y = 1.2*np.mean(θ_list)
y_bar = 2.
γ_1 = 1.7675/10000
γ_2 = .0022*2
γ_2p = np.array([0, .0197*2, .3853*2])
πd_o = np.array([1./3, 1./3, 1./3])

y_step = .02
y_grid_long = np.arange(0., 4., y_step)
y_grid_short = np.arange(0., 2+y_step, y_step)
n_bar = find_nearest_value(y_grid_long, y_bar) + 1

# Prepare ϕ conditional on low, high, extreme damage
model_res_list = []
for γ_2p_i in γ_2p:
    model_args = (η, δ, σ_y, y_bar, γ_1, γ_2, γ_2p_i, θ_list, πc_o, ξ_w, ξ_a) 
    model_res = solve_hjb_y(y_grid_long, model_args, v0=None, ϵ=1.,
                            tol=1e-8, max_iter=5_000, print_iteration=False)
    model_res_list.append(model_res)

ϕ_list = [res['v'] for res in model_res_list]
certainty_equivalent = -ξ_p*np.log(np.average(np.exp(-1./ξ_p*np.array(ϕ_list)), axis=0, weights=πd_o))
# Change grid from 0-4 to 0-2
ϕ_i = np.array([temp[:n_bar] for temp in ϕ_list])

# Compute ϕ with jump (impose boundary condition)
model_args = (η, δ, σ_y, y_bar, γ_1, γ_2, γ_2p, θ_list, πc_o, ϕ_i, πd_o, ξ_w, ξ_p, ξ_a)
model_res = solve_hjb_y_jump(y_grid_short, model_args, 
                             v0=np.average(ϕ_i, weights=πd_o, axis=0),
                             ϵ=1., tol=1e-8, max_iter=5_000, print_iteration=False)

Converged. Total iteration 5000: LHS Error: 2.544698815576396e-05; RHS Error 6.901213676188678e-05
Converged. Total iteration 1507: LHS Error: 9.965662961519683e-09; RHS Error 0.00047002516558698054
Converged. Total iteration 1621: LHS Error: 9.952296764481616e-09; RHS Error 0.002278795638726125
Converged. Total iteration 443: LHS Error: 9.762318953221438e-09; RHS Error 0.0008703085083172288


In [8]:
simulation_res = simulate_jump(model_res, θ_list)

In [33]:
years = np.arange(0,100)

s1 = figure(
    title="Value functions",
    x_axis_label='y',
    y_axis_label='ϕ(y)', 
    x_range=(0,3),
    plot_width=400, plot_height=400,
)
s1.line(model_res['y'], model_res['v'])
s1.multi_line(
    xs=[y_grid_long[n_bar:] for i in range(len(ϕ_list))], 
    ys=[ϕ_list[i][n_bar:] for i in range(len(ϕ_list))],
    line_color = ["darkgreen", "darkorange", "red"]
)
s1.line(y_grid_long[n_bar:], certainty_equivalent[n_bar:],
       line_dash="4 4")
jump_point = Span(location=y_bar, dimension='height', line_dash='dashed')
s1.add_layout(jump_point)



s2 = figure(
    title="Drift distortion", 
    x_axis_label='Years',
    y_axis_label='Emission', 
    y_range=(0,0.3),
    plot_width=400, plot_height=400
)
s2.line(years, simulation_res['ht'], line_color="navy")
s2.xgrid.grid_line_alpha=0.5


s3 = figure(
    title="Distorted probabilities of damages", 
    x_axis_label='Years', 
    y_axis_label='Emission', 
    plot_width=400, plot_height=400
)
s3.multi_line(
    xs = [years for i in range(3)], 
    ys=[simulation_res['πdt'][:,i] for i in range(3)], 
    line_color=["darkgreen", "darkorange", "red"]
)

show(row(s1,s3,s2))